In [ ]:
pip install ipfn


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
from ipfn import ipfn
import numpy as np
#a=pd.read_csv("/content/2D Ipf - Untitled spreadsheet - Sheet1.csv")
n=[[[476,410],[472,400],[407,483],[413,413]],[[410,400],[465,472],[460,488],[424,491]]]


xipp=np.array([4200,3446])
xpjp=np.array([2005,2000,1958,2043])
xppk=np.array([4250,4500])
xijp=np.array([[975,981,974,975],[946,952,947,946]])
xpjk=np.array([[1200,1183],[1000,950],[900,1050],[950,1109]])

n=np.array(n)
aggregates=[xipp,xpjp,xppk,xijp,xpjk]
dimensions=[[0],[1],[2],[0,1],[1,2]]
IPF=ipfn.ipfn(n,aggregates,dimensions,convergence_rate=1e-6)
n=IPF.iteration()
print(n)


ipfn converged: convergence_rate not updating or below rate_tolerance
[[[627.60051341 581.88855886]
  [529.46190017 460.16778445]
  [442.74658444 545.95587851]
  [500.92439278 544.11909901]]

 [[572.39948659 601.11144114]
  [470.53809983 489.83221555]
  [457.25341556 504.04412149]
  [449.07560722 564.88090099]]]
